In [1]:
# we are using pretrained model for image classication 
# list here : https://keras.io/api/applications/
# we prefer VGG16 : https://keras.io/api/applications/vgg/#vgg16-function
# we will use VGG16 model for image classification (light weight)

# it as combination of convolutional layers and dense layers
# convolutional layers are used to extract the features from the image
# dense layers are used to classify the image, based on the features extracted by convolutional layers

In [2]:
import os

In [3]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example\\research'

In [4]:
os.chdir('../')

In [5]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    "create configuration manager class to manage the configuration of the project"
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        '''
        This first part extracts the base model configuration from your YAML file 
        and ensures you have a directory to store your model files. 
        Think of it like preparing a workspace before assembling a complex piece of machinery.
        
        '''
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    
    
    '''
    Explanation of the above code : 
    
    root_dir: This is your workspace directory where all model-related files will be stored.
    base_model_path and updated_base_model_path: These are like "before" and "after" snapshots. The base_model_path is where your initial pre-trained model will be saved, and updated_base_model_path is where you'll save the model after modifying it for your specific task.
    params_image_size: This defines the dimensions your model expects for input images. If your model expects 224x224 pixel images, all your training images will need to match this size.
    params_learning_rate: This is like setting the "cautiousness" of your model during training. A higher learning rate means bigger adjustments during training, while a lower rate means more careful, smaller adjustments.
    params_include_top: This is a crucial flag that determines whether to include the classification layers of the original model. Setting this to False is common when you're adapting a pre-trained model for a new task.
    params_weights: This typically specifies whether to load pre-trained weights (like 'imagenet') or start from scratch.
    params_classes: This defines how many different categories your model needs to predict. In your case, this would be the number of different classifications in your chicken fecal image dataset.

    The type hint -> PrepareBaseModelConfig ensures that this method always returns the correct type of configuration object, making your code more reliable and easier to debug.
    
    '''

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        "downloading the VGG16 model here from keras library"
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        # below we have written a function for saving the model,  using it here
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    '''
    Explanation of the above code : 
    This method downloads VGG16, a powerful pre-trained model. Think of it like getting a skilled artist who has learned to recognize millions of images. The parameters are crucial:

    input_shape tells the model what size images to expect
    weights='imagenet' loads pre-learned knowledge from millions of images
    include_top=False removes the original classification layers since you'll add your own
    
    '''


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        "freeze_till is used to freeze the layers till the given layer, for ex, freeze 5 layers"
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        '''
        Explanation of the above code : 
        
        This part handles "transfer learning" by freezing layers. 
        When you freeze a layer, you're telling that part of the model "keep your knowledge,
        don't learn anything new." It's like telling an artist "keep your understanding of basic shapes and colors, 
        but we'll teach you specifically about chicken diseases."
        '''

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        '''
        Explanation of the above code : 
        This adds your custom classification head. The Flatten layer converts the complex image features into a simple format,
        and the Dense layer makes the final predictions for your specific classes.
        It's like adding a specialized chicken disease expert on top of the general artist's knowledge.
        
        '''

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        # if using keras, we have to compile the model
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    '''
    Explanation of the above code : 
    
    This sets up how the model will learn:

    SGD (Stochastic Gradient Descent) optimizer is like the model's learning strategy
    CategoricalCrossentropy loss helps the model understand how wrong its predictions are
    Accuracy metric tracks how well it's doing
    
    '''
    

    def update_base_model(self):
        '''
        This creates your final model architecture, freezing all the VGG16 layers 
        (freeze_all=True) 
        and adding your custom classification layers for chicken disease identification.
        '''
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        "function for saving the model"
        model.save(path)


In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

# it will start downloading the base VGG16 model from keras library
# after this, it will update the model,
# basically, it will add the custom layers instead of dense layers

# onc eexecuted you can check the artifcats folder, 
# you will see the base model and updated model being downloaded
# we will be using base_model_updated version for training the model on our dataset
# now convert this notebook as modular piece , go to config_entity.py


[2025-01-02 10:32:36,675: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 10:32:36,679: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 10:32:36,680: INFO: common: created directory at: artifacts]
[2025-01-02 10:32:36,684: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 23s 0us/step
[2025-01-02 10:33:01,268: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                